**This notebook is about different text classification methods and their comparsion. Done in learning purposes**

Let's import dataset:

In [130]:
import os
import os.path
import sklearn.feature_extraction.text as ftext
from sklearn import svm
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

In [131]:
dataset_dir = './spam'
fname = 'SMSSpamCollection.txt'

In [132]:
def isword(x):
    for l in x:
        if l.isdigit():
            return False
    return True

In [133]:
dataset = []
labels = []

with open(os.path.join(dataset_dir, fname), encoding='utf-8') as f:
    for line in f.readlines():
        tmp = line.split('\t')
        labels.append(0 if 'ham' in tmp[0] else 1)
        dataset.append(' '.join(list(filter(isword, tmp[1].split()))))

In [134]:
print('\n'.join(dataset[:10]))

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...
Free entry in a wkly comp to win FA Cup final tkts May Text FA to to receive entry question(std txt rate)T&C's apply
U dun say so early hor... U c already then say...
Nah I don't think he goes to usf, he lives around here though
FreeMsg Hey there darling it's been week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, to rcv
Even my brother is not like to speak with me. They treat me like aids patent.
As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press to copy your friends Callertune
WINNER!! As a valued network customer you have been selected to receivea prize reward! To claim call Claim code Valid hours only.
Had your mobile months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile U

**Naive bayes**

In [135]:
count = ftext.CountVectorizer()
tfidf = ftext.TfidfVectorizer()
mul_nb = MultinomialNB()

In [136]:
vectorizers = ['count', 'tfidf']

for i, vectorizer in enumerate((count, tfidf)):
    data_transformed = vectorizer.fit_transform(dataset)
    X_train, X_test, y_train, y_test = train_test_split(data_transformed, labels, test_size=0.35, random_state=0)
    mul_nb.fit(X_train, y_train)
    mul_nb.fit(X_train, y_train)
    y_pred = mul_nb.predict(X_test)
    print(f'{vectorizers[i]}\tf1-measure: {f1_score(y_test, y_pred)}\n{confusion_matrix(y_test, y_pred)}')

count	f1-measure: 0.9094202898550724
[[1650   32]
 [  18  251]]
tfidf	f1-measure: 0.7688787185354691
[[1682    0]
 [ 101  168]]


**TFIDF+PCA+SVM**

In [137]:
data_transformed = tfidf.fit_transform(dataset)
pca = PCA(n_components = 50)
data_pca = pca.fit_transform(data_transformed.toarray())

In [138]:
X_train, X_test, y_train, y_test = train_test_split(data_pca, labels, test_size=0.3, random_state=0)
svm_clf = svm.SVC(C=100000) ### this value of C shows the biggest model quality
svm_clf.fit(X_train, y_train)
y_pred = svm_clf.predict(X_test)

D:\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [139]:
print(f'f1-measure: {f1_score(y_test, y_pred)}\n{confusion_matrix(y_test, y_pred)}')

f1-measure: 0.9130434782608695
[[1423    8]
 [  32  210]]


**LSTM-network**

Necessary imports:

In [175]:
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn

In [176]:
lemmatizer = WordNetLemmatizer()
dataset_transformed = []

In [177]:
for sms in dataset:
    words = list(filter(isword, sms.split()))
    for idx, w in enumerate(words):
        words[idx] = lemmatizer.lemmatize(w)
        if any(x.isalpha() for x in words[idx]):
            words[idx] = words[idx].replace('.', '')
    dataset_transformed.append(nltk.word_tokenize(' '.join(words)))

In [178]:
maxlen = max([len(x) for x in dataset_transformed])
maxlen

209

In [179]:
vocab = set()
for sentence in dataset_transformed:
    for word in sentence:
        vocab.add(word)
    sentence += [''] * (maxlen - len(sentence))
vocab.add('')

In [184]:
word_to_idx = {}
for idx, val in enumerate(vocab):
    word_to_idx[val] = idx

In [185]:
dataset_transformed = np.array(dataset_transformed)